# Minimal Working Example for scBERT

**Install**
- make a conda environment
- add the dependincies [here](https://github.com/TencentAILabHealthcare/scBERT/blob/master/requirements.txt)
- note: not all dependencies will be resolved. `scanpy`, `pandas`, and `matplotlib` versions will need to be resolved manually
- hit 'em with a `pip install local-attention`
- clone the scBERT repo and point the path to include the `utils.py` and `perfomer_pytorch` directories

In [2]:
# Imports from the environment
import os
import gc
import argparse
import json
import random
import math
import random
from functools import reduce
import numpy as np
import pandas as pd
from scipy import sparse
from sklearn.model_selection import train_test_split, ShuffleSplit, StratifiedShuffleSplit, StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, precision_recall_fscore_support, classification_report
import torch
from torch import nn
from torch.optim import Adam, SGD, AdamW
from torch.nn import functional as F
# from performer_pytorch import PerformerLM
import scanpy as sc
import anndata as ad
# from utils import *
import pickle as pkl

In [3]:
# Imports from the scBERT repo
import sys
sys.path.append('/home/jpic/scFoundationProject/scBERT/scBERT/')
from scBERTutils import *
from performer_pytorch import PerformerLM

In [4]:
SEED            = 0    # args.seed
EPOCHS          = 10   # args.epoch
SEQ_LEN         = None # args.gene_num + 1
UNASSIGN        = None # args.novel_type
UNASSIGN_THRES  = None # args.unassign_thres if UNASSIGN == True else 0
CLASS           = None # args.bin_num + 2
POS_EMBED_USING = None # args.pos_embed
data_path       = '/nfs/turbo/umms-indikar/shared/projects/foundation_models/example_inputs/scBERT/Zheng68K.h5ad'

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'device={device}')

device=cuda


In [7]:
file_path = '/nfs/turbo/umms-indikar/shared/projects/foundation_models/scBERT/panglao_pretrain.pth'
data = torch.load(file_path)


In [9]:
if isinstance(data, dict):
    for key, value in data.items():
        print(f"Key: {key}")
        # print(f"Value: {value}\n")
else:
    print(data)


Key: epoch
Key: model_state_dict
Key: optimizer_state_dict
Key: scheduler_state_dict
Key: losses


In [8]:
adata = sc.read_h5ad(data_path)
adata

AnnData object with n_obs × n_vars = 68450 × 16906
    obs: 'TSNE.1', 'TSNE.2', 'celltype', 'n_genes'
    uns: 'log1p'

In [11]:
adata.X

<68450x16906 sparse matrix of type '<class 'numpy.float32'>'
	with 33627905 stored elements in Compressed Sparse Row format>

In [12]:
#load the label stored during the fine-tune stage
with open('label_dict', 'rb') as fp:
    label_dict = pkl.load(fp)
with open('label', 'rb') as fp:
    label = pkl.load(fp)

class_num = np.unique(label, return_counts=True)[1].tolist()
class_weight = torch.tensor([(1 - (x / sum(class_num))) ** 2 for x in class_num])
label = torch.from_numpy(label)
data = data.X


FileNotFoundError: [Errno 2] No such file or directory: 'label_dict'